### 交易策略
每天將所有股票進行推薦評分，然後將股票依照分數由高到低排名，交易分數高的前3名股票。
- 如果某支股票昨天是前3名，今天還是前3名，那這隻股票就不動作，繼續持有這支股票。
- 如果某支股票昨天不是前3名，今天不是前3名，那這支股票就不動作。
- 如果某支股票昨天不是前3名，今天是前3名，那今天就買入這支股票。
- 如果某支股票昨天是前3名，今天不是前3名，那今天就賣出這支股票。

如果今天有要踢除的股票，則會先執行賣出要被踢除的股票。\
接著才是將持有現金加上賣出股票的收入平均分成三等份，再執行買入新進股票。

交易股票代碼：["AAPL", "MSFT", "NVDA", "GOOGL", "AMZN", "META", "XOM", "TSLA", "AVGO"]\
初始資金設置為 2000000 元。\
訓練集數據範圍為 2020/1/1 至 2020/12/31，測試集數據範圍為 2021/1/1 至 2021/12/31。

### 載入所需套件

In [ ]:
import random

import gym
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import yfinance as yf
from stable_baselines3 import A2C

### 先準備訓練資料和測試資料

In [ ]:
# 使用 yfinance 下載股票的歷史資料，時間範圍為 2020-01-01 至 2021-12-31
# 訓練資料: 2020/1/1 ~ 2020/12/31
# 測試資料: 2021/1/1 ~ 2021/12/31

stocks = ["AAPL", "MSFT", "NVDA", "GOOGL", "AMZN", "META", "XOM", "TSLA", "AVGO"]
stock_mapping_float = {stock: float(i) for i, stock in enumerate(stocks)}

all_df_list = []
for stock in stocks:
    stock_df = (
        pd.DataFrame(yf.download(stock, start="2020-01-01", end="2021-12-31"))
        .droplevel("Ticker", axis=1)
        .reset_index()
        .ffill()
    )
    stock_df["Asset"] = stock
    all_df_list.append(stock_df)

all_df = pd.concat(all_df_list)
all_df["Asset"] = all_df["Asset"].map(stock_mapping_float)
train_df = all_df[all_df["Date"] <= "2020-12-31"]
test_df = all_df[all_df["Date"] > "2020-12-31"]

### 環境建置

In [5]:
class MultiStockTradingEnv(gym.Env):
    def __init__(self, df, stock_dim=9, top_k=3):
        """
        初始化多股票交易環境，設定初始參數和狀態
        df: 股票的歷史數據
        stock_dim: 股票數量，預設為9支股票
        top_k: 交易的前K個股票，預設為3
        """
        super(MultiStockTradingEnv, self).__init__()

        self.df = df  # 股票的歷史數據
        self.stock_dim = stock_dim  # 設定股票的數量，預設為9
        self.top_k = top_k  # 設定要交易的股票數量，預設為3
        self.initial_amount = 2000000  # 初始資金設為200萬
        self.buy_cost_pct = 0.001425  # 買入股票時的手續費比例
        self.sell_cost_pct = 0.001425 + 0.003  # 賣出股票時的手續費和證交稅
        self.reward_scaling = 1e-2  # Reward 的縮放(正規化)比例

        # 獲取所有交易日期，並按時間排序
        self.all_date = list(sorted(self.df["Date"].unique()))

        # 定義 Action 空間，表示推薦分數，範圍從0到1，大小為股票的數量
        self.action_space = gym.spaces.Box(
            low=0, high=1, shape=(stock_dim,), dtype=np.float32
        )

        # 定義 State 空間，State 包含現金、日期、持有的股票數量、股票代碼、股票價格數據(開高低收量)。
        # State可能數值範圍從0到無限大。
        # State[0]:現金； State[1]:日期；
        # State[2+i*7]:第i隻股票目前持有的股票數量； State[3+i*7]:第i隻股票代碼；
        # State[4+i*7]:第i隻股票開盤價； State[5+i*7]:第i隻股票最高價；
        # State[6+i*7]:第i隻股票最低價； State[7+i*7]:第i隻股票收盤價； State[8+i)*7]:第i隻股票交易量。
        self.observation_space = gym.spaces.Box(
            low=0, high=np.inf, shape=(2 + 7 * stock_dim,), dtype=np.float32
        )
        # 重置 Environment State 為初始 State
        self.reset()

    def reset(self):
        """
        重置環境狀態為初始狀態，通常在每個 Episode 的開始時調用。
        """
        # 設置隨機種子
        random.seed(7777)
        np.random.seed(7777)
        torch.manual_seed(7777)

        self.day = 0  # 初始化執行天數為第一天
        # 取出第一天歷史資料當作初始股票價格數據
        self.data = self.df[self.df["Date"] == self.all_date[self.day]]

        # 初始化現金、日期和股票狀態
        state = [self.initial_amount, self.day]
        for stock in self.data["Asset"].unique():
            stock_data = self.data[self.data["Asset"] == stock].iloc[0]
            state += [0, stock] + stock_data[
                ["Open", "High", "Low", "Close", "Volume"]
            ].tolist()
        self.state = np.array(state, dtype=np.float32)

        self.reward = 0  # 初始化累積收益為0。

        # self.terminal 的值用來判斷交易是否應該結束，如果是 True 就是一個 Episode 結束
        # 重置終止標誌為 False，表示新的一輪交易開始
        self.terminal = False

        self.asset_memory = [self.initial_amount]  # 初始化資產記錄為初始資金
        self.actions_memory = []  # 初始化 Action 記錄
        self.date_memory = [self.all_date[self.day]]  # 初始化日期記錄為第一天日期
        self.trade_memory = []  # 初始化交易記錄
        self.last_topk = []  # 初始化前一天的 topk 清單
        return self.state

    def step(self, actions):
        """
        根據 Action 來更新環境狀態，計算回報，並判斷 Episode 是否結束。
        """
        # 檢查是否到達資料的最後一天，若到達最後一天，則設置 terminal 為 True
        self.terminal = self.day >= len(self.all_date) - 1
        if self.terminal:
            return self.state, self.reward, self.terminal, {}

        # 計算今天開始時的總資產(現金+持有股票的價值)
        begin_total_asset = self.state[0]
        for i in range(self.stock_dim):
            begin_total_asset += (
                self.state[4 + i * 7] * self.state[2 + i * 7]
            )  # 開盤價 * 持有股票數量

        # 根據 Action 來選擇今天的 top_k 股票
        sorted_indices = np.argsort(actions)[::-1][: self.top_k]
        current_topk = [
            self.state[3 + i * 7] for i in sorted_indices
        ]  # 3 + i*7: 股票代碼

        # 賣出昨天的 top_k 股票，若今天不再是 top_k
        sell_indices = [
            i
            for i in range(self.stock_dim)
            if self.state[3 + i * 7] in self.last_topk
            and self.state[3 + i * 7] not in current_topk
        ]
        for i in sell_indices:
            sell_num_shares = self.state[2 + i * 7]  # 2 + i*7: 股票數量
            if sell_num_shares > 0:
                # 計算賣出股票後獲得的現金，扣除賣出手續費
                sell_amount = (
                    sell_num_shares * self.state[4 + i * 7] * (1 - self.sell_cost_pct)
                )
                self.state[0] += sell_amount  # 更新現金餘額
                self.state[2 + i * 7] -= sell_num_shares  # 更新持有股票數量
                # 記錄賣出交易的日期、股票代碼、類型、數量和金額
                self.trade_memory.append(
                    (
                        self.all_date[self.day],
                        self.state[3 + i * 7],
                        "sell",
                        sell_num_shares,
                        sell_amount,
                    )
                )

        available_cash = self.state[0]  # 計算剩餘現金

        # 買入新的 top_k 股票
        buy_indices = [
            i for i in sorted_indices if self.state[3 + i * 7] not in self.last_topk
        ]  # 3 + i*7: 股票代碼
        if buy_indices:
            # 計算每支股票可以分配的現金
            cash_per_stock = available_cash / len(buy_indices)
            for i in buy_indices:
                # 計算可以買入的股票數量，根據分配的現金和買入價格
                buy_num_shares = cash_per_stock // (
                    self.state[4 + i * 7] * (1 + self.buy_cost_pct)
                )
                if buy_num_shares > 0:
                    # 計算買入股票所需的現金，包含買入手續費
                    buy_amount = (
                        buy_num_shares * self.state[4 + i * 7] * (1 + self.buy_cost_pct)
                    )
                    self.state[0] -= buy_amount  # 更新現金餘額
                    self.state[
                        2 + i * 7
                    ] += buy_num_shares  # 2 + i*7: 股票數量 # 更新持有股票數量
                    # 記錄買入交易的日期、股票代碼、類型、數量和金額
                    self.trade_memory.append(
                        (
                            self.all_date[self.day],
                            self.state[3 + i * 7],
                            "buy",
                            buy_num_shares,
                            buy_amount,
                        )
                    )

        # 更新到下一天
        self.day += 1
        self.data = self.df[self.df["Date"] == self.all_date[self.day]]
        state = [self.state[0], self.day]  # 更新狀態，包含現金和日期
        for stock in self.data["Asset"].unique():
            stock_data = self.data[self.data["Asset"] == stock].iloc[
                0
            ]  # 取得該股票的新數據
            # 更新持有股票數量和新的股票價格數據
            state += [
                self.state[2 + self.data["Asset"].tolist().index(stock) * 7],
                stock,
            ] + stock_data[["Open", "High", "Low", "Close", "Volume"]].tolist()
        self.state = np.array(state, dtype=np.float32)

        # 計算今天結束時的總資產
        end_total_asset = self.state[0]
        for i in range(self.stock_dim):
            end_total_asset += (
                self.state[4 + i * 7] * self.state[2 + i * 7]
            )  # 開盤價 * 持有股票數量
        self.reward = (end_total_asset - begin_total_asset) * self.reward_scaling

        # 記錄資產、日期和 Action 的變化
        self.asset_memory.append(end_total_asset)
        self.date_memory.append(self.all_date[self.day])
        self.actions_memory.append(actions)

        # 更新前一天的topk清單
        self.last_topk = current_topk
        # 回傳狀態、回報、是否終止和其他信息
        return self.state, self.reward, self.terminal, {}

    def render(self, mode="human"):
        """顯示交易環境的當前狀態。"""
        print(f"目前日期: {self.all_date[self.day]}")
        print(f"目前執行到第幾天: {self.day}")
        print(f"目前Reward: {self.reward}")
        print(f"目前現金金額: {self.state[0]}")
        print(
            f"目前總資產: {self.state[0] + sum(self.state[4 + i*7] * self.state[2 + i*7] for i in range(self.stock_dim))}"
        )

    def save_asset_memory(self):
        """儲存每一天的總資產記錄，並回傳一個包含日期和資產的 DataFrame。"""
        return pd.DataFrame({"date": self.date_memory, "asset": self.asset_memory})

    def save_action_memory(self):
        """儲存每一天的 Action 記錄，並回傳一個包含日期和動作的 DataFrame。"""
        return pd.DataFrame(
            {"date": self.date_memory[:-1], "actions": self.actions_memory}
        )

    def save_trade_memory(self):
        """儲存發生交易的記錄，並回傳一個包含日期、交易類型、交易股票數量和交易價格的 DataFrame。"""
        return pd.DataFrame(
            self.trade_memory, columns=["date", "asset", "type", "shares", "amount"]
        )

In [ ]:
# 使用訓練數據集 train_df 創建交易環境
env = MultiStockTradingEnv(train_df)

# 使用 A2C 算法創建模型
model = A2C("MlpPolicy", env, verbose=1)

# 訓練模型，總訓練步數為 10000 步
model.learn(total_timesteps=10000)

# 將訓練好的模型儲存為文件 A2C_multi_stock.zip
model.save("A2C_multi_stock")

# 載入儲存的模型
model = A2C.load("A2C_multi_stock")

# 評估模型套在訓練集的結果
obs = env.reset()
for i in range(len(all_df["Date"].unique())):
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()

# 儲存交易結果
asset_memory = env.save_asset_memory()  # 儲存資產變化的記錄
asset_memory.to_csv("A2C_multi_stock_assets.csv")

action_memory = env.save_action_memory()  # 儲存 Action 記錄
action_memory.to_csv("A2C_multi_stock_actions.csv")

trade_memory = env.save_trade_memory()  # 儲存交易記錄
trade_memory.to_csv("A2C_multi_stock_trades.csv")

In [ ]:
# 畫出資產變化圖
plt.plot(asset_memory["date"], asset_memory["asset"])
plt.xlabel("Date")
plt.ylabel("Asset Value")
plt.title("Cumulative Asset Value over Time (PPO)")
plt.xticks(rotation=45)
plt.show()

In [ ]:
# 顯示交易記錄
# 顯示買入進場的時間點
buy_df = trade_memory[(trade_memory["type"] == "buy")]
buy_df = buy_df[(trade_memory["shares"] != 0)]
print("買入進場的時間點:")
print(buy_df)

# 顯示賣出出場的時間點
sell_df = trade_memory[(trade_memory["type"] == "sell")]
sell_df = sell_df[(trade_memory["shares"] != 0)]
print("賣出出場的時間點:")
print(sell_df)

In [ ]:
train_df["Date"]

In [ ]:
env = MultiStockTradingEnv(all_df)
# 載入儲存的模型
model = A2C.load("A2C_multi_stock")
# 評估模型套在訓練集的結果
obs = env.reset()
for i in range(len(all_df["Date"].unique())):
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()
# 儲存交易結果
asset_memory = env.save_asset_memory()  # 儲存資產變化的記錄
action_memory = env.save_action_memory()  # 儲存 Action 記錄
trade_memory = env.save_trade_memory()  # 儲存交易記錄

split_index = len(asset_memory) // 2
first_half = asset_memory.iloc[:split_index]
second_half = asset_memory.iloc[split_index:]

plt.plot(first_half["date"], first_half["asset"], color="black", label="First Half")
plt.plot(second_half["date"], second_half["asset"], color="red", label="Second Half")
# plt.plot(asset_memory["date"], asset_memory["asset"])
plt.xlabel("Date")
plt.ylabel("Asset Value")
plt.title("Cumulative Asset Value over Time (PPO)")
plt.xticks(rotation=45)
plt.show()